In [ ]:
import os
import sys
from typing import List

import numpy as np
import pandas as pd


os.environ["JAVA_HOME"] = "<insert path>"
os.environ["SPARK_HOME"] = "<insert path>"
os.environ["HADOOP_CONF_DIR"] = "<insert path>"
os.environ["PYSPARK_PYTHON"] = "<insert path>"
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], "python", "lib", "pyspark.zip"))
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], "python", "lib", "py4j-0.10.9.7-src.zip"))

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import Window

In [ ]:
conf = (SparkConf()
    .setAll([
        ("spark.executor.cores", "2"),
        ("spark.executor.memory", "15G"),
        ("spark.dynamicAllocation.enabled", "true"),
        ("spark.dynamicAllocation.maxExecutors", "10"),
        ("spark.dynamicAllocation.cachedExecutorIdleTimeout", "30m"),
        ("spark.shuffle.service.enabled", "true"),
        ("spark.sql.execution.arrow.pyspark.enabled", "true"),
        ("spark.sql.catalogImplementation", "hive"),
    ])
)

In [ ]:
spark = (SparkSession.builder
    .master("yarn")
    .appName("spark-uniform-shuffle")
    .config(conf=conf)
    .getOrCreate()
)

In [ ]:
KEY_TYPE = "integer"

TYPE_MAPPING = {
    "long": "int64",
    "integer": "int32",
    "short": "int16",
}

def get_key_list(spark: SparkSession,
                 num_keys: int,
                 key_type: str = KEY_TYPE) -> List[int]:
    """
    Generating key list for uniform shuffle
    Генерация списка ключей для равномерного перемешивания

    Arguments:
    spark: SparkSession object
    num_keys: Number of desired keys for shuffle
    key_type: Any type of integer containing values of result list

    Returns:
    key_list: List of generated keys
    """
    win_spec = (Window
        .partitionBy("mod")
        .orderBy("id")
    )
    
    key_list = (spark
        .range(1_000_000, numPartitions=2)
        .select(
            F.col("id").cast(key_type)
        )
        .select(
            F.col("id"),
            
            F.when(
                F.hash("id") % num_keys >= 0,                 
                F.hash("id") % num_keys
            ).otherwise(
                F.hash("id") % num_keys + num_keys
            ).alias("mod"),
        )
        .select(
            F.col("id"),
            F.row_number().over(win_spec).alias("rn"),
        )
        .where(
            F.col("rn") == 1
        )
        .rdd.map(
            lambda row: row["id"]
        )
        .collect()
    )

    return key_list

### Объяснение

##### Блок кода 1
```python
.range(1_000_000, numPartitions=2)
.select(
    F.col("id").cast(key_type)
)
```
Значение hash-функции в spark зависит от типа аргумента и вычисляется по алгоритму Murmur3 (имплементирован в классе `org.apache.spark.unsafe.hash.Murmur3_x86_32`), поэтому необходимо зафиксировать тип генерируемой последовательности "на берегу". Тип `key_type` должен вмещать значения последовательности из выражения `spark.range`

In [7]:
Murmur3_x86_32 = spark._jvm.org.apache.spark.unsafe.hash.Murmur3_x86_32

In [8]:
Murmur3_x86_32.hashInt(101, 42)

-818933188

In [9]:
(spark
    .createDataFrame(
        [(101,)],
        schema="id integer"
    )
    .select(
        F.col("id"),
        F.hash("id"),
    )
    .show(1, False)
)

+---+----------+
|id |hash(id)  |
+---+----------+
|101|-818933188|
+---+----------+



##### Блок кода 2
```python
F.when(
    F.hash("id") % num_keys >= 0,                 
    F.hash("id") % num_keys
).otherwise(
    F.hash("id") % num_keys + num_keys
).alias("mod")
```
Значение хэш-функции является беззнаковым 4-х байтным целым числом (`uint32`), в то время как Spark поддерживает только знаковые 4-х байтные целые числа (`int32`), поэтому результат хэш-функции может быть отрицательным из-за переполнения разрядов `int32`.  
Вычисление остатка от деления в Java не является беззнаковой операцией, т.к. не является строго арифметической, как в Python, поэтому поле `mod`, задуманное как остаток от деления результата хэш-функции на число партиций, требует добавления числа партиций в случае отрицательного значения хэш-функции.

##### Блок кода 3
```python
win_spec = (Window
    .partitionBy("mod")
    .orderBy("id")
)

.select(
    F.col("id"),
    F.row_number().over(win_spec).alias("rn"),
)
.where(
    F.col("rn") == 1
)
```
Отбор по одному значению поля `id` на каждое уникальное значение поля `mod`, т.о. формируется результирующий список сгенерированных значений для равномерного шаффлинга